#Ml-Projects-1(Credit Transfer)


Import Dataset using API

In [3]:
!pip install kaggle

In [7]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle datasets download -d mlg-ulb/creditcardfraud


Dataset URL: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud
License(s): DbCL-1.0
 64% 42.0M/66.0M [00:00<00:00, 148MB/s]
100% 66.0M/66.0M [00:00<00:00, 170MB/s]


In [11]:
from zipfile import ZipFile

dataset="/content/creditcardfraud.zip"

with ZipFile (dataset,'r') as zip:
  zip.extractall()
  print("Finish extracting")

Finish extracting


#Load dependancies(Libraries)

In [13]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [14]:
df=pd.read_csv("/content/creditcard.csv")

Dataset information

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [15]:
df.shape

(284807, 31)

In [17]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
df.isnull().sum()

#No null values found

Distribution

In [29]:
df["Class"].value_counts()

# Unbalance dataset(Huge different with fraud and legit)

Class
0    284315
1       492
Name: count, dtype: int64

1--> Fraud transiction

0--> Legit transiction

Seperating data for analysis

In [23]:
legit = df[df.Class==0]
fraud = df[df.Class==1]

In [25]:
print(legit.shape)
print(fraud.shape)

(284315, 31)
(492, 31)


In [26]:
legit.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [27]:
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

Compare Class data

In [28]:
df.groupby("Class").mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


#Under-sampling for this scenario

In [43]:
legit_sample = legit.sample(n=492)

In [44]:
print(legit_sample)

            Time        V1        V2        V3        V4        V5        V6  \
61054    49634.0 -0.597921  0.146442  2.356114 -0.202912 -0.297851 -0.055555   
128730   78816.0  1.452938 -0.476693  0.224378 -0.851749 -0.724304 -0.560904   
258137  158510.0  2.016771 -0.329583 -1.712320  0.261810  0.336801 -0.414749   
68372    52960.0 -1.113017  1.050028  1.764830  0.611791  0.045280  0.003381   
20631    31168.0  0.719337  1.559219 -1.624824  1.771940  0.787594 -1.746675   
...          ...       ...       ...       ...       ...       ...       ...   
111110   72095.0  0.441400 -1.294986  0.270252  1.045673 -1.100985 -0.283134   
204040  135084.0 -0.843449  0.371714  0.074943 -1.774453  2.410617  4.029540   
109353   71346.0 -1.531968  1.148306  1.404001  1.566547  0.647913 -0.863690   
122506   76590.0  0.702022 -0.475371  0.430480  1.686640 -0.789953 -0.156000   
22356    32200.0  1.125919 -0.704209 -0.478228 -1.378714 -0.613539 -1.012586   

              V7        V8        V9  .

In [45]:
legit_sample.shape

(492, 31)

Concate the legit and fraud dataframe

In [46]:
target = pd.concat([legit_sample,fraud],axis=0)

In [47]:
target.head()

#here target is our new dataset

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
61054,49634.0,-0.597921,0.146442,2.356114,-0.202912,-0.297851,-0.055555,0.426080,-0.224236,1.116411,...,0.008917,0.450544,0.071902,0.405406,-1.058866,-0.034198,-0.398892,-0.225660,32.71,0
128730,78816.0,1.452938,-0.476693,0.224378,-0.851749,-0.724304,-0.560904,-0.560766,-0.141536,-0.706217,...,-0.148478,-0.553458,0.024964,-0.481210,0.341664,-0.428197,0.008469,0.015170,14.96,0
258137,158510.0,2.016771,-0.329583,-1.712320,0.261810,0.336801,-0.414749,0.145028,-0.121081,1.077328,...,-0.260152,-0.767763,0.167400,0.019877,0.026829,-0.497671,-0.038414,-0.048413,50.06,0
68372,52960.0,-1.113017,1.050028,1.764830,0.611791,0.045280,0.003381,0.451565,0.082909,-0.075191,...,-0.218171,-0.223881,-0.238809,0.127902,0.285629,0.489408,0.296598,0.210894,19.67,0
20631,31168.0,0.719337,1.559219,-1.624824,1.771940,0.787594,-1.746675,0.802483,-0.110579,-0.205136,...,-0.098227,-0.052831,0.190915,0.227022,-0.645216,-0.400362,0.266789,0.014327,0.70,0


In [48]:
target.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1
281674,170348.0,1.991976,0.158476,-2.583441,0.408670,1.151147,-0.096695,0.223050,-0.068384,0.577829,...,-0.164350,-0.295135,-0.072173,-0.450261,0.313267,-0.289617,0.002988,-0.015309,42.53,1


In [49]:
target["Class"].value_counts()

Class
0    492
1    492
Name: count, dtype: int64

In [50]:
target.groupby("Class").mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,92259.532520,0.061302,0.068212,0.051259,0.089557,-0.020090,0.014469,-0.009805,0.067113,-0.029605,...,-0.004319,0.002852,0.047261,-0.014144,0.03211,0.011047,0.008001,0.004673,0.000703,80.759228
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.10513,0.041449,0.051648,0.170575,0.075667,122.211321


Splitting the data into X and Y

In [51]:
X = target.drop(columns="Class",axis=1)
Y = target.Class

In [52]:
print(X)

            Time        V1        V2        V3        V4        V5        V6  \
61054    49634.0 -0.597921  0.146442  2.356114 -0.202912 -0.297851 -0.055555   
128730   78816.0  1.452938 -0.476693  0.224378 -0.851749 -0.724304 -0.560904   
258137  158510.0  2.016771 -0.329583 -1.712320  0.261810  0.336801 -0.414749   
68372    52960.0 -1.113017  1.050028  1.764830  0.611791  0.045280  0.003381   
20631    31168.0  0.719337  1.559219 -1.624824  1.771940  0.787594 -1.746675   
...          ...       ...       ...       ...       ...       ...       ...   
279863  169142.0 -1.927883  1.125653 -4.518331  1.749293 -1.566487 -2.010494   
280143  169347.0  1.378559  1.289381 -5.004247  1.411850  0.442581 -1.326536   
280149  169351.0 -0.676143  1.126366 -2.213700  0.468308 -1.120541 -0.003346   
281144  169966.0 -3.113832  0.585864 -5.399730  1.817092 -0.840618 -2.943548   
281674  170348.0  1.991976  0.158476 -2.583441  0.408670  1.151147 -0.096695   

              V7        V8        V9  .

In [53]:
print(Y)

61054     0
128730    0
258137    0
68372     0
20631     0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64


In [54]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=2,stratify=Y)

#Stratify = class value evenly split hobe

In [55]:
print(X.shape, X_train.shape, X_test.shape)

(984, 30) (787, 30) (197, 30)


#Model training
Logistic Regression

In [64]:
logistic_regression = LogisticRegression()

In [65]:
logistic_regression.fit(X_train, Y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

#Accuracy Score

In [67]:
X_train_prediction = logistic_regression.predict(X_train)
training_accuracy = accuracy_score(X_train_prediction,Y_train)

In [74]:
print("Training data accuracy:",training_accuracy)

Training data accuracy: 0.9504447268106735


#Accuracy for testing data

In [72]:
X_test_prediction = logistic_regression.predict(X_test)
testing_accuracy = accuracy_score(X_test_prediction, Y_test)

In [75]:
print("Testing data accuracy:",testing_accuracy)

Testing data accuracy: 0.9289340101522843
